# DECISION TREE, GAUSSIAN NB, KNEIGHBORS Y RANDOM FOREST
Proyecto: BNP Paribas Cardif Claims Management

Blanca Ubaque - 201725992		
Carolina Trespalacios - 200313688		
Santiago Barco - 200913781		
Rafael Niño - 200522388		

In [1]:
import pandas as pd
import numpy as np

# Lectura de la data
X_train = pd.read_csv('X_dtrain.csv', sep=',',na_values=[''],header=0)
y_train = pd.read_csv('y_dtrain.csv', sep=',',na_values=[''],header=0)
X_test = pd.read_csv('X_dtest.csv', sep=',',na_values=[''],header=0)
y_test = pd.read_csv('y_dtest.csv', sep=',',na_values=[''],header=0)

In [2]:
X_train.drop('Unnamed: 0', inplace=True, axis=1)
X_test.drop('Unnamed: 0', inplace=True, axis=1)
y_train.drop('Unnamed: 0', inplace=True, axis=1)
y_test.drop('Unnamed: 0', inplace=True, axis=1)

# Selección de variables

In [3]:
# Selección de variables por stepwise

import pandas as pd
import numpy as np
import statsmodels.api as sm

X = X_train
y = y_train

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X, y)

print('resulting features:')
print(result)

/Users/santiagobarcogomez/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/santiagobarcogomez/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/santiagobarcogomez/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/santiagobarcogomez/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/santiagobarcogomez/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encount

Add  v50                            with p-value 0.0
Add  v110_B                         with p-value 2.44631e-233
Add  v47_C                          with p-value 2.44631e-233
Add  v66_C                          with p-value 2.44246e-95
Add  v66_B                          with p-value 1.08257e-43
Add  v66_A                          with p-value 0.0
Add  v14                            with p-value 1.74369e-24
Add  v24_E                          with p-value 1.53245e-12
Add  v79_O                          with p-value 2.19915e-13
Add  v119                           with p-value 2.15096e-12
Add  v31_C                          with p-value 2.80786e-11
Add  v24_C                          with p-value 1.36304e-07
Add  v79_J                          with p-value 1.83698e-06
Add  v47_E                          with p-value 0.000192752
Add  v79_D                          with p-value 0.000192752
Add  v47_D                          with p-value 0.00045017
Add  v112_F                         wit

In [4]:
# Se dejan las variables que resultaron del stepwise
var=['v50', 'v47_C', 'v66_C', 'v110_B', 'v66_B', 'v14', 'v66_A', 'v24_E', 'v79_O', 'v119', 'v31_C', 'v24_C', 'v79_J', 'v47_E', 'v79_D', 'v47_D', 'v112_F', 'v74_C', 'v79_A', 'v31_B']
X_train=X_train[var]
X_test=X_test[var]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Se crea un diccionario para facilitar la estimación de los modelos requeridos
models = {'dt': DecisionTreeClassifier(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier()}

In [6]:
for model in models.keys():
    models[model].fit(X_train, y_train.values.ravel())

In [7]:
# predict test for each model
y_pred = pd.DataFrame(index=y_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

y_pred

,dt,nb,nn
0,1,0,1
1,0,0,1
2,1,0,1
3,1,1,1
4,0,0,1
5,1,1,1
6,0,0,0
7,1,1,1
8,1,0,1
9,1,0,1


In [8]:
# Se calcula el accuracy de los modelos sobre la muestra de test
from sklearn import metrics

for model in models.keys():
    print(model,metrics.accuracy_score(y_pred[model], y_test))

dt 0.681836966629
nb 0.52567413989
nn 0.732100423597


In [9]:
# Se generan las matrices de confusión
from sklearn.metrics import confusion_matrix

for model in models.keys():
    print(confusion_matrix(y_test, y_pred[model]))

[[ 1722  2984]
 [ 3175 11477]]
[[3865  841]
 [8341 6311]]
[[ 1170  3536]
 [ 1650 13002]]


In [10]:
# Se evalúa cada modelo con el F1 Score
from sklearn.metrics import f1_score
for model in models.keys():
    print(model, f1_score(y_test, y_pred[model]))

dt 0.788445024559
nb 0.578884608329
nn 0.833728759218


In [11]:
# Se calcula el AUC de cada modelo
from sklearn.metrics import roc_auc_score
for model in models.keys():
    print(model, roc_auc_score(y_test, y_pred[model]))

dt 0.574610942705
nb 0.626009074213
nn 0.568003085959


# Random Forest

In [12]:
# Se modela un random forest classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
print(clf.feature_importances_)

[ 0.44108799  0.12033335  0.06117952  0.11577947  0.05961185  0.08861259
  0.          0.          0.          0.00487683  0.0030743   0.          0.
  0.0183858   0.          0.          0.          0.          0.          0.0870583 ]


In [14]:
# Se predice con random forest con la muestra de test
y_pred = clf.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [15]:
# Se calcula el accuracy del modelo
print(metrics.accuracy_score(y_pred, y_test))

0.756896373592


In [16]:
# Se calcula el AUC del modelo
print(roc_auc_score(y_test, y_pred))

0.5


# Crass Validation

## CV - Decision Tree Classifier

In [17]:
from sklearn.cross_validation import cross_val_score
tree = DecisionTreeClassifier()
results_tree = cross_val_score(tree, X_train, y_train.values.ravel(), cv=20, scoring='roc_auc')

/Users/santiagobarcogomez/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
pd.Series(results_tree).plot()

In [19]:
pd.Series(results_tree).describe()

count    20.000000
mean      0.573886
std       0.013273
min       0.549160
25%       0.567306
50%       0.571571
75%       0.583139
max       0.601462
dtype: float64

## CV - Gaussian NB

In [20]:
from sklearn.cross_validation import cross_val_score
GNB = DecisionTreeClassifier()
results_GNB = cross_val_score(GNB, X_train, y_train.values.ravel(), cv=20, scoring='roc_auc')

In [21]:
pd.Series(results_GNB).plot()

In [22]:
pd.Series(results_GNB).describe()

count    20.000000
mean      0.574095
std       0.011449
min       0.552128
25%       0.567104
50%       0.574979
75%       0.580417
max       0.594312
dtype: float64

## CV - K Neighbors Classifier

In [23]:
from sklearn.cross_validation import cross_val_score
KNN = KNeighborsClassifier()
results_KNN = cross_val_score(KNN, X_train, y_train.values.ravel(), cv=20, scoring='roc_auc')

In [24]:
pd.Series(results_KNN).plot()

In [25]:
pd.Series(results_KNN).describe()

count    20.000000
mean      0.645689
std       0.011746
min       0.625641
25%       0.635833
50%       0.646248
75%       0.652440
max       0.669458
dtype: float64

## CV - Random Forest Classifier

In [26]:
from sklearn.cross_validation import cross_val_score
RF = RandomForestClassifier()
results_RF = cross_val_score(RF, X_train, y_train.values.ravel(), cv=20, scoring='roc_auc')

In [27]:
pd.Series(results_RF).plot()

In [28]:
pd.Series(results_RF).describe()

count    20.000000
mean      0.665881
std       0.013108
min       0.640793
25%       0.662695
50%       0.665045
75%       0.671027
max       0.688934
dtype: float64